In [44]:
import warnings
warnings.filterwarnings('ignore')

import sys
import random
import numpy as np
import pyodbc 
from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#Conectando com o Impala
pyodbc.autocommit = True
conn_impala = pyodbc.connect('DSN=Sample Cloudera Impala DSN 64', autocommit=True)
cursor_impala = conn_impala.cursor()

In [ ]:
cursor_impala.execute('show databases; ')

In [ ]:
def as_pandas(cursor_impala):
    names = [metadata[0] for metadata in cursor_impala.description]
    return pd.DataFrame([dict(zip(names, row)) for row in cursor_impala], columns=names)

In [ ]:
as_pandas(cursor_impala)

In [ ]:
cursor_impala.execute('use voos_brasil_labdata; ')

In [ ]:
cursor_impala.execute('show tables; ')
as_pandas(cursor_impala)

In [ ]:
cursor_impala.execute('select * from voos_historico_tratada_5 where situacao_voo = "REALIZADO"')# where situacao_voo in (%s)', (['REALIZADO'],))

In [2]:
dataset = pd.read_csv('voos.csv', encoding='utf-8', sep=',')#as_pandas(cursor_impala)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
dataset

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero
0,AZU,4059,0,N,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,SBPA,3,36.0,1002.3,26.2,21.4,63.0,1.0,N,SBKP,VIRACOPOS,CAMPINAS,SP,BRASIL,AMERICA DO SUL,2018-02-19 12:35:00.0,2018,2,19,2018-02-19,1235,2018-02-19 12:27:00.0,2018.0,2.0,19.0,SBKP,661.0,14.0,922.8,22.5,19.3,73.0,1.0,SE,2018-02-19 10:55:00.0,2018.0,2.0,19.0,2018-02-19,1235,2018-02-19 10:51:00.0,REALIZADO,HD,0,-4.0,0,-8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,36,1002.3,26.2,21.4,63,1,661,14,922.8,22.5,19.3,73,1,3,36,1002.3,26.2,21.4,63.0,1.0,661.0,14.0,922.80,22.5,19.30,73.0,1.00,3,36,1002.3,26.2,21.4,63,1,661,14,922.8,22.5,19.3,73,1
1,AZU,2828,0,R,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,SBPA,3,36.0,1002.3,26.2,21.4,63.0,1.0,N,SBPK,PELOTAS,PELOTAS,RS,BRASIL,AMERICA DO SUL,2018-02-19 12:20:00.0,2018,2,19,2018-02-19,1220,2018-02-19 12:20:00.0,2018.0,2.0,19.0,SBPK,13.0,5.0,1005.0,25.6,23.0,79.0,1.0,NE,2018-02-19 11:25:00.0,2018.0,2.0,19.0,2018-02-19,1220,2018-02-19 11:25:00.0,REALIZADO,NaN,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,36,1002.3,26.2,21.4,63,1,13,5,1005.0,25.6,23.0,79,1,3,36,1002.3,26.2,21.4,63.0,1.0,13.0,5.0,1005.00,25.6,23.00,79.0,1.00,3,36,1002.3,26.2,21.4,63,1,13,5,1005.0,25.6,23.0,79,1
2,AZU,2765,0,R,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,SBPA,3,36.0,1002.3,26.2,21.4,63.0,1.0,N,SBSP,CONGONHAS,S�O PAULO,SP,BRASIL,AMERICA DO SUL,2018-02-19 13:10:00.0,2018,2,19,2018-02-19,1310,2018-02-19 12:56:00.0,2018.0,2.0,19.0,SBSP,803.0,14

In [5]:
# PANDAS: Verifica a dimensão dos dados (linhas, colunas)
dim = dataset.shape


In [8]:
dim

(99999, 113)

In [9]:
# PANDAS: Verifica os tipos de dados de cada atributo
tipos = dataset.dtypes
print(tipos)

icao                                         object
num_voo                                       int64
cod_aut_di                                   object
cod_tipo_linha                               object
cod_aero_org                                 object
aeroporto_origem                             object
cidade_origem                                object
uf_origem                                    object
pais_origem                                  object
continente_origem                            object
icao_origem                                  object
altitudeicao_origem                           int64
direcaovento_origem                         float64
pressaoatmestacao_origem                    float64
tempbulboseco_origem                        float64
tempbulboumido_origem                       float64
umidaderelativa_origem                      float64
velocidadeventonebulosidade_origem          float64
descricao_cod_origem                         object
cod_aero_des

In [10]:
# PANDAS: Estatística descritiva
pd.set_option('display.width', 100)
pd.set_option('precision', 15)
resultado = dataset.describe()
print(resultado)

                     num_voo    altitudeicao_origem    direcaovento_origem  \
count  99999.000000000000000  99999.000000000000000  91934.000000000000000   
mean    4338.648446484465239    135.075790757907583      8.341560249744381   
std     2179.979406798576747    260.402041294402238     10.379229609241170   
min        2.000000000000000      1.000000000000000      0.000000000000000   
25%     2575.000000000000000      1.000000000000000      0.000000000000000   
50%     3882.000000000000000      6.000000000000000      5.000000000000000   
75%     5865.000000000000000     28.000000000000000     14.000000000000000   
max     9998.000000000000000    999.000000000000000     36.000000000000000   

       pressaoatmestacao_origem   tempbulboseco_origem  tempbulboumido_origem  \
count     99990.000000000000000  99992.000000000000000  96437.000000000000000   
mean        954.165062506334948     23.308602688214954     19.150183021040135   
std          56.065141581061461      4.356384536222087

In [11]:
dataset.head(10)

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero
0,AZU,4059,0,N,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,SBPA,3,36.0,1002.299999999999955,26.199999999999999,21.399999999999999,63.0,1.0,N,SBKP,VIRACOPOS,CAMPINAS,SP,BRASIL,AMERICA DO SUL,2018-02-19 12:35:00.0,2018,2,19,2018-02-19,1235,2018-02-19 12:27:00.0,2018.0,2.0,19.0,SBKP,661.0,14.0,922.799999999999955,22.500000000000000,19.300000000000001,73.0,1.0,SE,2018-02-19 10:55:00.0,2018.0,2.0,19.0,2018-02-19,1235,2018-02-19 10:51:00.0,REALIZADO,HD,0,-4.0,0,-8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,36,1002.299999999999955,26.199999999999999,21.399999999999999,63,1,661,14,922.799999999999955,22.500000000000000,19.300000000000001,73,1,3,36,1002.299999999999955,26.199999999999999,21.399999999999999,63.0,1.0,661.0,14.0,922.799999999999955,22.500000000000000,19.300000000000001,73.0,1.0,3,36,1002.299999999999955,26.199999999999999,21.399999999999999,63,1,661,14,922.799999999999955,22.500000000000000,19.300000000000001,73,1
1,AZU,2828,0,R,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,SBPA,3,36.0,1002.299999999999955,26.199999999999999,21.399999999999999,63.0,1.0,N,SBPK,PELOTAS,PELOTAS,RS,BRASIL,AMERICA DO SUL,2018-02-19 12:20:00.0,2018,2,19,2018-02-19,1220,2018-02-19 12:20:00.0,2018.0,2.0,19.0,SBPK,13.0,5.0,1005.000000000000000,25.600000000000001,23.000000000000000,79.0,1.0,NE,2018-02-19 11:25:00.0,2018.0,2.0,19.0,2018-02-19,1220,2018-02-19 11:25:00.0,REALIZADO,NaN,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,36,1002.29999999999

In [12]:
#TRATAMENTOS DE DADO
dataset_filtrado = dataset.loc[dataset['situacao_voo'] == 'REALIZADO']

In [101]:
#DUMMIES
dummies_aero_org = pd.get_dummies(dataset_filtrado['cod_aero_org'], prefix='dummy')
dummies_aero_dest = pd.get_dummies(dataset_filtrado['cod_aero_dest'], prefix='dummy')

colunas_dummie_aero_org = list(dummies_aero_org)
colunas_dummie_aero_dest = list(dummies_aero_dest)

dataset_filtrado = pd.concat([dataset_filtrado, dummies_aero_org], axis=1)
dataset_filtrado = pd.concat([dataset_filtrado, dummies_aero_dest], axis=1)

In [66]:
dataset_filtrado

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero
0,AZU,4059,0,N,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,SBPA,3,36.0,1002.299999999999955,26.199999999999999,21.399999999999999,63.0,1.0,N,SBKP,VIRACOPOS,CAMPINAS,SP,BRASIL,AMERICA DO SUL,2018-02-19 12:35:00.0,2018,2,19,2018-02-19,1235,2018-02-19 12:27:00.0,2018.0,2.0,19.0,SBKP,661.0,14.0,922.799999999999955,22.500000000000000,19.300000000000001,73.0,1.0,SE,2018-02-19 10:55:00.0,2018.0,2.0,19.0,2018-02-19,1235,2018-02-19 10:51:00.0,REALIZADO,HD,0,-4.0,0,-8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,36,1002.299999999999955,26.199999999999999,21.399999999999999,63,1,661,14,922.799999999999955,22.500000000000000,19.300000000000001,73,1,3,36,1002.299999999999955,26.199999999999999,21.399999999999999,63.0,1.0,661.0,14.0,922.799999999999955,22.500000000000000,19.300000000000001,73.0,1.00,3,36,1002.299999999999955,26.199999999999999,21.399999999999999,63,1,661,14,922.799999999999955,22.500000000000000,19.300000000000001,73,1
1,AZU,2828,0,R,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,SBPA,3,36.0,1002.299999999999955,26.199999999999999,21.399999999999999,63.0,1.0,N,SBPK,PELOTAS,PELOTAS,RS,BRASIL,AMERICA DO SUL,2018-02-19 12:20:00.0,2018,2,19,2018-02-19,1220,2018-02-19 12:20:00.0,2018.0,2.0,19.0,SBPK,13.0,5.0,1005.000000000000000,25.600000000000001,23.000000000000000,79.0,1.0,NE,2018-02-19 11:25:00.0,2018.0,2.0,19.0,2018-02-19,1220,2018-02-19 11:25:00.0,REALIZADO,NaN,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,36,1002.2999999999

In [67]:
# PANDAS: Verifica a dimensão dos dados (linhas, colunas)
dim = dataset_filtrado.shape
print(dim)

(99999, 113)


In [68]:
# PANDAS: Estatística descritiva
pd.set_option('display.width', 100)
pd.set_option('precision', 15)
resultado = dataset_filtrado.describe()
print(resultado)

                     num_voo    altitudeicao_origem    direcaovento_origem  \
count  99999.000000000000000  99999.000000000000000  91934.000000000000000   
mean    4338.648446484465239    135.075790757907583      8.341560249744381   
std     2179.979406798576747    260.402041294402238     10.379229609241170   
min        2.000000000000000      1.000000000000000      0.000000000000000   
25%     2575.000000000000000      1.000000000000000      0.000000000000000   
50%     3882.000000000000000      6.000000000000000      5.000000000000000   
75%     5865.000000000000000     28.000000000000000     14.000000000000000   
max     9998.000000000000000    999.000000000000000     36.000000000000000   

       pressaoatmestacao_origem   tempbulboseco_origem  tempbulboumido_origem  \
count     99990.000000000000000  99992.000000000000000  96437.000000000000000   
mean        954.165062506334948     23.308602688214954     19.150183021040135   
std          56.065141581061461      4.356384536222087

In [69]:
# PANDAS: Distribuição das classses
contagem = dataset_filtrado.groupby('flg_atraso_partida').size()
print(contagem)

flg_atraso_partida
0    95011
1     4988
dtype: int64


In [70]:
# PANDAS: Correlação
cor = dataset_filtrado.corr(method='pearson')
print(cor)

                                                    num_voo  altitudeicao_origem  \
num_voo                                   1.000000000000000    0.072189418226146   
altitudeicao_origem                       0.072189418226146    1.000000000000000   
direcaovento_origem                       0.059717157988970    0.202921752820305   
pressaoatmestacao_origem                  0.234033797676208    0.002764356851204   
tempbulboseco_origem                      0.009225038889074    0.040509473306024   
tempbulboumido_origem                     0.002349174288690    0.022725415697083   
umidaderelativa_origem                    0.047801473147921   -0.023887930908611   
velocidadeventonebulosidade_origem       -0.048856431857736    0.006329570678896   
chegada_prevista_ano                      0.233857267486334    0.261409765754277   
chegada_prevista_mes                      0.148084849127713    0.073022062695627   
chegada_prevista_dia                     -0.006977803295825   -0.01896906127

In [71]:
dataset_filtrado.shape

(99999, 113)

In [102]:
dataset_2017 = dataset_filtrado.loc[dataset_filtrado['partida_prevista_ano'] == 2017]
dataset_2018 = dataset_filtrado.loc[dataset_filtrado['partida_prevista_ano'] == 2018]


In [97]:
dataset_2017.shape

(7732, 113)

In [98]:
dataset_2018.shape

(1153, 113)

In [100]:
dataset_2017

,icao,num_voo,cod_aut_di,cod_tipo_linha,cod_aero_org,aeroporto_origem,cidade_origem,uf_origem,pais_origem,continente_origem,icao_origem,altitudeicao_origem,direcaovento_origem,pressaoatmestacao_origem,tempbulboseco_origem,tempbulboumido_origem,umidaderelativa_origem,velocidadeventonebulosidade_origem,descricao_cod_origem,cod_aero_dest,aeroporto_dest,cidade_dest,uf_dest,pais_dest,continente_dest,chegada_prevista_completa,chegada_prevista_ano,chegada_prevista_mes,chegada_prevista_dia,chegada_prevista_date,chegada_prevista_hora,chegada_real,chegada_real_ano,chegada_real_mes,chegada_real_dia,icao_dest,altitudeicao_dest,direcaovento_dest,pressaoatmestacao_dest,tempbulboseco_dest,tempbulboumido_dest,umidaderelativa_dest,velocidadeventonebulosidade_dest,descricao_cod_dest,partida_prevista_completa,partida_prevista_ano,partida_prevista_mes,partida_prevista_dia,partida_prevista_date,partida_prevista_hora,partida_real,situacao_voo,cod_justificativa,flg_atraso_partida,diff_minutos_partida,flg_atraso_chegada,diff_minutos_chegada,feriado_federal_partida,feriado_federal_chegada,feriado_estadual_partida,feriado_estadual_chegada,feriado_municipal_partida,feriado_municipal_chegada,feriado_facultativo_partida,feriado_facultativo_chegada,carnaval_partida,carnaval_chegada,dia_da_semana_partida,dia_da_semana_chegada,final_de_semana_partida,final_de_semana_chegada,altitudeicao_origem_moda,direcaovento_origem_moda,pressaoatmestacao_origem_moda,tempbulboseco_origem_moda,tempbulboumido_origem_moda,umidaderelativa_origem_moda,velocidadeventonebulosidade_origem_moda,altitudeicao_dest_moda,direcaovento_dest_moda,pressaoatmestacao_dest_moda,tempbulboseco_dest_moda,tempbulboumido_dest_moda,umidaderelativa_dest_moda,velocidadeventonebulosidade_dest_moda,altitudeicao_origem_media,direcaovento_origem_media,pressaoatmestacao_origem_media,tempbulboseco_origem_media,tempbulboumido_origem_media,umidaderelativa_origem_media,velocidadeventonebulosidade_origem_media,altitudeicao_dest_media,direcaovento_dest_media,pressaoatmestacao_dest_media,tempbulboseco_dest_media,tempbulboumido_dest_media,umidaderelativa_dest_media,velocidadeventonebulosidade_dest_media,altitudeicao_origem_zero,direcaovento_origem_zero,pressaoatmestacao_origem_zero,tempbulboseco_origem_zero,tempbulboumido_origem_zero,umidaderelativa_origem_zero,velocidadeventonebulosidade_origem_zero,altitudeicao_dest_zero,direcaovento_dest_zero,pressaoatmestacao_dest_zero,tempbulboseco_dest_zero,tempbulboumido_dest_zero,umidaderelativa_dest_zero,velocidadeventonebulosidade_dest_zero
2647,AZU,2485,0,N,SBPK,PELOTAS,PELOTAS,RS,BRASIL,AMERICA DO SUL,SBPK,13,18.0,1011.000000000000000,25.800000000000001,24.000000000000000,86.0,2.0,S,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,2017-01-02 14:00:00.0,2017,1,2,2017-01-02,1400,2017-01-02 13:58:00.0,2017.0,1.0,2.0,SBPA,3.0,14.0,1005.799999999999955,28.000000000000000,25.800000000000001,83.0,1.0,SE,2017-01-02 13:10:00.0,2017.0,1.0,2.0,2017-01-02,1400,2017-01-02 13:12:00.0,REALIZADO,HD,1,2.0,0,-2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,18,1011.000000000000000,25.800000000000001,24.000000000000000,86,2,3,14,1005.799999999999955,28.000000000000000,25.800000000000001,83,1,13,18,1011.000000000000000,25.800000000000001,24.000000000000000,86.0,2.0,3.0,14.0,1005.799999999999955,28.000000000000000,25.800000000000001,83.0,1.0,13,18,1011.000000000000000,25.800000000000001,24.000000000000000,86,2,3,14,1005.799999999999955,28.000000000000000,25.800000000000001,83,1
2648,AZU,2485,0,N,SBPK,PELOTAS,PELOTAS,RS,BRASIL,AMERICA DO SUL,SBPK,13,14.0,1016.700000000000045,22.600000000000001,18.600000000000001,67.0,2.0,SE,SBPA,SALGADO FILHO,PORTO ALEGRE,RS,BRASIL,AMERICA DO SUL,2017-01-06 14:00:00.0,2017,1,6,2017-01-06,1400,2017-01-06 13:56:00.0,2017.0,1.0,6.0,SBPA,3.0,14.0,1011.100000000000023,24.800000000000001,21.600000000000001,74.0,3.0,SE,2017-01-06 13:10:00.0,2017.0,1.0,6.0,2017-01-06,1400,2017-01-06 13:06:00.0,REALIZADO,HD,0,-4.0,0,-4.0,0,0,0,0,0,0,0,0,0,0,4,4,1,1,13,14,1016.70000000000

In [104]:
#cols = ['direcaovento_origem','altitudeicao_origem','umidaderelativa_origem','velocidadeventonebulosidade_origem','chegada_prevista_ano','chegada_prevista_mes','chegada_prevista_dia','chegada_prevista_hora','chegada_real_ano','chegada_real_mes','chegada_real_dia','altitudeicao_dest','direcaovento_dest','umidaderelativa_dest','velocidadeventonebulosidade_dest','partida_prevista_ano','partida_prevista_mes','partida_prevista_dia','partida_prevista_hora']

cols = ['feriado_federal_chegada','altitudeicao_origem_moda', 'direcaovento_origem_moda', 'pressaoatmestacao_origem_moda', 'tempbulboseco_origem_moda', 'tempbulboumido_origem_moda'] + colunas_dummie_aero_dest + colunas_dummie_aero_org

train_y =  dataset_2017['flg_atraso_partida']
train_x =  dataset_2017[cols]

test_y = dataset_2018['flg_atraso_partida']
test_x = dataset_2018[cols]

In [105]:
#REGRESSÃO LOGÍSTICA

clf_lr = linear_model.LogisticRegression(penalty='l2')
clf_lr.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [106]:
predictions = clf_lr.predict(test_x)

In [107]:
print(confusion_matrix(test_y,predictions))

[[995   0]
 [158   0]]


In [111]:
print(classification_report(test_y,predictions))

             precision    recall  f1-score   support

          0       0.86      1.00      0.93       995
          1       0.00      0.00      0.00       158

avg / total       0.74      0.86      0.80      1153



In [112]:
print(accuracy_score(test_y,predictions))

0.8629661751951431


In [113]:
#RANDOM FOREST

rfc = RandomForestClassifier(max_depth=30, random_state=42)
rfc.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [114]:
predictions = rfc.predict(test_x)

In [115]:
print(confusion_matrix(test_y,predictions))

[[971  24]
 [155   3]]


In [116]:
print(classification_report(test_y,predictions))

             precision    recall  f1-score   support

          0       0.86      0.98      0.92       995
          1       0.11      0.02      0.03       158

avg / total       0.76      0.84      0.79      1153



In [117]:
print(accuracy_score(test_y,predictions))

0.8447528187337381


In [121]:
print (sorted(zip(map(lambda x: round(x, 4), rfc.feature_importances_), cols), 
             reverse=True))

[(0.3052, 'pressaoatmestacao_origem_moda'), (0.2896, 'tempbulboseco_origem_moda'), (0.2442, 'tempbulboumido_origem_moda'), (0.0914, 'direcaovento_origem_moda'), (0.0055, 'feriado_federal_chegada'), (0.0047, 'dummy_SBCD'), (0.0042, 'dummy_SBCT'), (0.0041, 'dummy_SBPA'), (0.0041, 'altitudeicao_origem_moda'), (0.0035, 'dummy_SBJU'), (0.0022, 'dummy_SBFZ'), (0.0022, 'dummy_SBCB'), (0.0021, 'dummy_SBGV'), (0.0021, 'dummy_SBCD'), (0.0019, 'dummy_SWPI'), (0.0018, 'dummy_SBPK'), (0.0017, 'dummy_SBBR'), (0.0013, 'dummy_SBLO'), (0.0013, 'dummy_SBGL'), (0.0012, 'dummy_SBLE'), (0.0011, 'dummy_SBSM'), (0.001, 'dummy_SBJV'), (0.001, 'dummy_SBGU'), (0.001, 'dummy_SBGL'), (0.0009, 'dummy_SBKP'), (0.0006, 'dummy_SBCX'), (0.0005, 'dummy_SBNM'), (0.0005, 'dummy_SBLJ'), (0.0004, 'dummy_SBSR'), (0.0003, 'dummy_SBRF'), (0.0003, 'dummy_SBPF'), (0.0003, 'dummy_SBAX'), (0.0002, 'dummy_SBSR'), (0.0002, 'dummy_SBPA'), (0.0002, 'dummy_SBMQ'), (0.0002, 'dummy_SBLE'), (0.0002, 'dummy_SBCF'), (0.0002, 'dummy_SBBE'),